In [1]:
!rm -r smart-cities-drl
!git clone https://github.com/eescriba/smart-cities-drl
!cd smart-cities-drl/ && pip install -e .
!pip install ray[rllib]

rm: cannot remove 'smart-cities-drl': No such file or directory
Cloning into 'smart-cities-drl'...
remote: Enumerating objects: 595, done.
remote: Counting objects: 100% (595/595), done.
remote: Compressing objects: 100% (418/418), done.
remote: Total 595 (delta 212), reused 513 (delta 142), pack-reused 0
Receiving objects: 100% (595/595), 875.81 KiB | 5.21 MiB/s, done.
Resolving deltas: 100% (212/212), done.
Obtaining file:///content/smart-cities-drl
     |████████████████████████████████| 675kB 4.3MB/s 
     |████████████████████████████████| 61kB 7.6MB/s 
     |████████████████████████████████| 61kB 7.3MB/s 
  Running setup.py develop for smart-cities-drl
     |████████████████████████████████| 49.7MB 63kB/s 
     |████████████████████████████████| 81kB 9.4MB/s 
     |████████████████████████████████| 133kB 39.4MB/s 
     |████████████████████████████████| 1.3MB 38.6MB/s 
     |████████████████████████████████| 1.0MB 40.0MB/s 
     |████████████████████████████████| 81kB 9.7MB/s 
  

In [3]:
import json
import shutil

import gym
import ray
import ray.rllib.agents.ppo as ppo

import sys
sys.path.insert(0,'./smart-cities-drl/src/')
from wastenet.env import WasteNetEnv

In [4]:
ray.shutdown()
ray.init(ignore_reinit_error=True)

CHECKPOINT_ROOT = "tmp/ppo/wastenet"
shutil.rmtree(CHECKPOINT_ROOT, ignore_errors=True, onerror=None)

config = ppo.DEFAULT_CONFIG.copy()
config["num_gpus"] = 1
config["horizon"]=100

agent = ppo.PPOTrainer(config, env=WasteNetEnv)

N_ITER = 30
results = []
episode_data = []
episode_json = []

for n in range(N_ITER):
    result = agent.train()
    results.append(result)
    
    episode = {'n': n, 
               'episode_reward_min': result['episode_reward_min'], 
               'episode_reward_mean': result['episode_reward_mean'], 
               'episode_reward_max': result['episode_reward_max'],  
               'episode_len_mean': result['episode_len_mean']
              }
    
    episode_data.append(episode)
    episode_json.append(json.dumps(episode))
    file_name = agent.save(CHECKPOINT_ROOT)
    
    print(f'{n+1:3d}: Min/Mean/Max reward: {result["episode_reward_min"]:8.4f}/{result["episode_reward_mean"]:8.4f}/{result["episode_reward_max"]:8.4f}, len mean: {result["episode_len_mean"]:8.4f}. Checkpoint saved to {file_name}')

2021-04-27 17:37:51,359	INFO services.py:1269 -- View the Ray dashboard at http://127.0.0.1:8265
(raylet) /usr/local/lib/python3.7/dist-packages/ray/autoscaler/_private/cli_logger.py:61: FutureWarning: Not all Ray CLI dependencies were found. In Ray 1.4+, the Ray CLI, autoscaler, and dashboard will only be usable via `pip install 'ray[default]'`. Please update your install command.
(raylet)   "update your install command.", FutureWarning)
(raylet) /usr/local/lib/python3.7/dist-packages/ray/autoscaler/_private/cli_logger.py:61: FutureWarning: Not all Ray CLI dependencies were found. In Ray 1.4+, the Ray CLI, autoscaler, and dashboard will only be usable via `pip install 'ray[default]'`. Please update your install command.
(raylet)   "update your install command.", FutureWarning)
(pid=3048) WARNING:tensorflow:From /usr/local/lib/python3.7/dist-packages/tensorflow/python/compat/v2_compat.py:96: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will b

RayTaskError(ValueError): ignored

In [ ]:
policy = agent.get_policy()
model = policy.model
print(model.base_model.summary())

In [ ]:
obs = env.reset()
done = False
episode_reward = 0
sum_reward = 0
n_step = 20
for step in range(n_step):
    action = agent.compute_action(obs)
    print(action)
    state, reward, done, info = env.step(action)
    print(state, reward, done, info)
    sum_reward += reward
    if done:
        print("cumulative reward", sum_reward)
        state = env.reset()
        sum_reward = 0